In [1]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

"\nimport ibmos2spark\n# @hidden_cell\ncredentials = {\n    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',\n    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',\n    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',\n    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'\n}\n\nconfiguration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'\ncos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')\n\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.getOrCreate()\n\ndf = spark.read  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')  .option('header', 'true')  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))\n"

In [2]:
from functions import *
from datetime import datetime

In [3]:
timeSignature = str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
images_dir = "images/ML"
models_dir = "models"
logs_dir = models_dir+"/"+ timeSignature +"/logs.md"


file = "data/US_Accidents_Dec19.csv"
filter_state = 'PA'

df, sc,spark = setup_spark(file)
createDirectory(models_dir+"/"+timeSignature +"/")

In [4]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [5]:
printSparkConf(sc, logs_dir)

Config: [('spark.driver.extraClassPath', '/opt/conda/lib/python3.7/site-packages/sparkmonitor/listener.jar'), ('SPARKMONITOR_UI_HOST', '192.168.1.228'), ('spark.app.id', 'local-1588640602873'), ('spark.executor.id', 'driver'), ('spark.driver.host', 'localhost'), ('spark.app.name', 'Spark Project'), ('spark.executor.cores', '4'), ('executor-memory', '6'), ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), ('spark.num.executors', '6'), ('spark.kryoserializer.buffer.max', '10'), ('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.memory.fraction', '.6'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.port', '41849')]


# Debug tool

In [6]:
df,_ = df.randomSplit([1.0,0.0], 1) # Speed up the program a bit
df.count()

2974335

# 1. Define variables

In [7]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

df, colCat, colNum = setup_variables(df, sc, colLabel, colRem, logs_dir)  

write2file(logs_dir,"Number of rows", str(df.count()))

Label: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'] 
Categories: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 2. Preprocessing

In [8]:
from pyspark.ml.feature import VectorAssembler,OneHotEncoderEstimator,QuantileDiscretizer, StringIndexer, Imputer,StandardScaler,MinMaxScaler
from pyspark.ml import Pipeline
from numpy import argmax

## Analyse single state
This could be CA since it stands for a little less than 50% of the total set

In [9]:
if filter_state != '':
    df = df.filter(df.State == filter_state) # Lowers the dataset quite a lot
    write2file(logs_dir,"Specified state",str(filter_state))
else:
    write2file(logs_dir,"No state specified","")

## Modify time
Convert the timestamp into a numeric value and then into a string so that the time of day and month can be categorised.

In [10]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)
write2file(logs_dir,"Dataset after modifying UTC timestamp", str(df.take(1)))

+-----+--------+-------------------+---------+---------+------------+------+------+----+-------+------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+
|  TMC|Severity|         Start_Time|Start_Lat|Start_Lng|Distance(mi)|Number|Street|Side|   City|County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station| Stop|Traffic_Calming|Traffic_Signal|Turning_Loop|Sunrise_Sunset|Ci

## Quantiles function
Alternative solution to get position 
Remove City, Country, State, Zipcode, Airport_Code
In order to reduce dimensionality feature crossing will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data

In [11]:
# TODO - Calculate how many buckets needed (Bins)
discretizer_Lat = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lat", outputCol="Start_Lat_disc")
discretizer_Lng = QuantileDiscretizer(numBuckets=100, inputCol="Start_Lng", outputCol="Start_Lng_disc")

# Into categorical values
indexer_cord = [StringIndexer(inputCol=c + "_disc", outputCol=c+"_IDX") for c in ["Start_Lat","Start_Lng"]]

# One-hot (feature crossing)
encoder_cord = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexer_cord], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexer_cord])

pipeline_cord = Pipeline(stages=[discretizer_Lat,discretizer_Lng, *indexer_cord, encoder_cord])
preprocessed_cord = pipeline_cord.fit(df).transform(df)

position = VectorAssembler(inputCols=["Start_Lat_IDX_vec","Start_Lng_IDX_vec"], outputCol="position")

df = position.transform(preprocessed_cord)

In [12]:
df.show(1)
write2file(logs_dir,"Dataset after adding quantiles",str(df.take(1)))

+-----+--------+-------------------+---------+---------+------------+------+------+----+-------+------+-----+-------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+----------+-----------+------------+-------------+--------------+--------------+-------------+-------------+-----------------+-----------------+--------------------+
|  TMC|Severity|         Start_Time|Start_Lat|Start_Lng|Distance(mi)|Number|Street|Side|   City|County|State|Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|Amenity| Bump|Crossing|Give_

## Cast data 
Cast all the datacolumns into correct format so they will be sorted to numerical or categorical values. Data removed caused to big feature vector, contained too many NaN values or were converted into another form.

In [13]:
df = df.select(
        #col('TMC').cast('string'),  Lot of missing values
        col('Severity').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        #col('position'), # already preprocessed
        col('Distance(mi)').cast('double'),
        col('City').cast('string'),
        col('State').cast('string'), #col('County').cast('string'), 
        col('Side').cast('string'),   
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'), #  Lot of missing values
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'),  #Lot of missing values
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

# Since the last check can be unorganised we recreate a new list that contains all data
#df.cache()  # TODO does this work? - Heatbeat crash otherwise
colLabel = ["Severity"]

colCat, colNum = createNewClasses(df, sc, colLabel, logs_dir)

Label: ['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'City', 'State', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'] 
Categories: ['Start_Hour', 'Start_Month', 'Weather_Hour', 'Weather_Month', 'City', 'State', 'Side', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Clean data

### Recheck the missing values
Check so that the output contains 0 missing values

In [14]:
printMissingValues(df, logs_dir)

Missing values[Row(Severity=0, Start_Hour=0, Start_Month=0, Weather_Hour=798, Weather_Month=798, Distance(mi)=0, City=0, State=0, Side=0, Temperature(F)=1207, Wind_Chill(F)=53527, Humidity(%)=1343, Pressure(in)=864, Visibility(mi)=1308, Wind_Direction=985, Wind_Speed(mph)=17354, Weather_Condition=1274, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=0, Civil_Twilight=0, Nautical_Twilight=0, Astronomical_Twilight=0)]


### Remove data with occurance less than 1%
Based on information from analysis. With further analysis lower procentage can be used to find better results.

In [15]:
original_rows = df.count()
n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False)
df_filtered = weather_freq.filter(weather_freq['count'] > n)
filtered_conditions = df_filtered.select("Weather_Condition").rdd.flatMap(lambda x: x).collect()
df = df.filter(df['Weather_Condition'].isin(*filtered_conditions))

df_filtered.show()
print("Rows removed:",original_rows - df.count())

write2file(logs_dir,"Weather condition", str(df_filtered.take(df_filtered.count())) + "\nRows removed: " + str(original_rows - df.count()))

+-------------------+-----+
|  Weather_Condition|count|
+-------------------+-----+
|              Clear|28687|
|           Overcast|18185|
|      Mostly Cloudy| 9406|
|               Fair| 6915|
|   Scattered Clouds| 6179|
|      Partly Cloudy| 5010|
|         Light Rain| 4833|
|             Cloudy| 3356|
|         Light Snow| 2095|
|               null| 1274|
|               Rain|  996|
|                Fog|  941|
|               Haze|  706|
|         Heavy Rain|  375|
|               Snow|  257|
|      Light Drizzle|  189|
|Light Freezing Rain|  165|
|         Heavy Snow|   91|
+-------------------+-----+

Rows removed: 2009


## Prepare Pipeline

In [16]:
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features",handleInvalid="keep")
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol="num_features", outputCol="scaledFeatures")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='keep') for c in colCat]

encoder = OneHotEncoderEstimator(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_vec".format(indexer.getOutputCol()) for indexer in indexers],handleInvalid="keep")

## Preprocessing - Pipeline

In [17]:
# Categorical values

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

pipeline = Pipeline(stages=[numPipeline, catPipeline])
preprocessed_df = pipeline.fit(df).transform(df)

In [18]:
preprocessed_df.select("scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(1,False)

+---------------------------------------------------------------------------------------------------------------------------------------------+------------------+-------------------+--------------------+---------------------+------------------+-------------+-------------+----------------------+-------------------------+---------------+-------------+----------------+----------------+----------------+---------------+---------------+------------------+---------------+-------------+-----------------------+----------------------+--------------------+----------------------+----------------------+-------------------------+-----------------------------+
|scaledFeatures                                                                                                                               |Start_Hour_IDX_vec|Start_Month_IDX_vec|Weather_Hour_IDX_vec|Weather_Month_IDX_vec|City_IDX_vec      |State_IDX_vec|Side_IDX_vec |Wind_Direction_IDX_vec|Weather_Condition_IDX_vec|Amenity_IDX_vec|Bump_IDX_

## Combine features

In [19]:
va2 = VectorAssembler(inputCols=["scaledFeatures",*[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

df = va2.transform(preprocessed_df)

df = df.withColumn('label', col("Severity"))
df_features = df.withColumn('features', df.final_features).select("features","label")
df_features.show(1, False)

write2fileModel(df_features, models_dir,"df_features", timeSignature)

#df_features.write.format('parquet').mode('overwrite').option("header", "true").save(models_dir+'/df_features.parquet')
write2file(logs_dir,"Feature set size", str(df_features.count()) + "\n\n__Feature vector and label:__\n" + str(df_features.take(1)))
write2file(logs_dir,"Number of rows", str(df_features.count()))
print("Feature set size: ",df_features.count())

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                             |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Count categorical values

In [20]:
# Checking how many classes that can be used
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in colCat] 
print("Unique column values:", tmp)

write2file(logs_dir,"Unique column values", str(tmp))

Unique column values: [Row(Start_Hour=24), Row(Start_Month=12), Row(Weather_Hour=24), Row(Weather_Month=12), Row(City=1086), Row(State=1), Row(Side=2), Row(Wind_Direction=24), Row(Weather_Condition=17), Row(Amenity=2), Row(Bump=2), Row(Crossing=2), Row(Give_Way=2), Row(Junction=2), Row(No_Exit=2), Row(Railway=2), Row(Roundabout=1), Row(Station=2), Row(Stop=2), Row(Traffic_Calming=2), Row(Traffic_Signal=2), Row(Turning_Loop=1), Row(Sunrise_Sunset=2), Row(Civil_Twilight=2), Row(Nautical_Twilight=2), Row(Astronomical_Twilight=2)]


# 3. Feature importance

In [21]:
from pyspark.ml.feature import ChiSqSelector,PCA

## PCA
In order to understand how much the variance affect the dataset we check with PCA. Try to get above 90%

In [22]:
k=250
pca = PCA(k=k, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df_features)
pca_df = pca_model.transform(df_features)
#chi_model[0].pca_features

print("PCA - Feature Variance:","Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+str(k)+"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
write2file(logs_dir,"PCA - Feature variance", "Top 50:\n" + str(pca_model.explainedVariance[:50]) + "\nNumber of items: "+ str(k) +"\nSum of variance: "+ str(sum(pca_model.explainedVariance)))
write2fileModel(pca_df, models_dir, "pca_df", timeSignature)

PCA - Feature Variance: Top 50:
[0.1084606  0.04887023 0.0391047  0.03223598 0.02727851 0.02479509
 0.0236579  0.02274469 0.02203286 0.02121568 0.02080695 0.01960652
 0.01874708 0.01795997 0.01717408 0.01686945 0.01646196 0.01543653
 0.01534089 0.01507955 0.01492685 0.01365635 0.01281599 0.01157016
 0.01104464 0.01076146 0.01072034 0.01003153 0.00962655 0.0094986
 0.00923856 0.00881244 0.00865981 0.00839613 0.00786474 0.00773318
 0.00750525 0.00730812 0.00700776 0.00680787 0.00658201 0.00636836
 0.00630263 0.00600014 0.00588419 0.00568452 0.00540776 0.00526724
 0.00505954 0.00503982]
Number of items: 250
Sum of variance: 0.9803030484952425
Saving into: models/2020-05-05_01-03-20/pca_df
Saving through format 1


## ChiSqSelector
Check top 100 which of the values in the feature vector

In [23]:
selector = ChiSqSelector(numTopFeatures=k, 
                         labelCol='label', 
                         featuresCol='features', 
                         outputCol="selectedFeatures",
                         selectorType='numTopFeatures', 
                         percentile=0.1, 
                         fpr=0.05, fdr=0.05, fwe=0.05)

chi_model = selector.fit(df_features)
chi_df = chi_model.transform(df_features)

write2file(logs_dir,"Top selected features according to ChiSqSelector", str(chi_model.selectedFeatures)+ "\nNumber of features: " + str(k) + "\nExample data:\n"+str(chi_df.take(5)))
write2fileModel(chi_df, models_dir, "chi_df", timeSignature)

print("Top selected features according to ChiSqSelector:", chi_model.selectedFeatures)
print("Transformed selected features:",chi_df.head().selectedFeatures)
print("Transformed feature vector:", chi_df.show(10))

Saving into: models/2020-05-05_01-03-20/chi_df
Saving through format 1
Top selected features according to ChiSqSelector: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 16, 24, 25, 26, 27, 28, 29, 30, 32, 45, 46, 47, 53, 55, 62, 63, 64, 65, 66, 67, 68, 70, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 97, 99, 100, 101, 102, 103, 104, 105, 106, 110, 111, 113, 114, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127, 129, 130, 131, 132, 133, 135, 137, 141, 145, 149, 156, 157, 161, 162, 164, 168, 169, 170, 171, 172, 178, 179, 183, 185, 187, 197, 198, 203, 207, 208, 213, 216, 220, 222, 223, 225, 229, 230, 233, 234, 238, 242, 249, 252, 256, 258, 264, 266, 268, 272, 275, 277, 280, 283, 285, 298, 300, 315, 324, 329, 332, 333, 343, 358, 361, 365, 368, 369, 374, 382, 386, 396, 413, 414, 424, 432, 442, 448, 457, 458, 460, 476, 479, 486, 524, 551, 556, 566, 580, 604, 605, 612, 614, 628, 636, 663, 678, 721, 745, 1172, 1173, 1175, 1200, 1202, 1205, 1206, 1208, 1209, 1212, 1215, 1218, 1219, 1224, 1225, 1230,

# 4. Machine learning

In [24]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
# Use the chi_df for training since it is reduced
#trainSet, testSet = df_features.randomSplit([0.8 ,0.2], 1)
trainSet, testSet = chi_df.randomSplit([0.8 ,0.2], 1)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [26]:
trainSet.show(2,False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                       

## Logistic regression

In [27]:
lr = LogisticRegression(labelCol="label", featuresCol="selectedFeatures")

paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.01]) \
                .addGrid(lr.maxIter, [10]) \
                .addGrid(lr.elasticNetParam, [0.6]) \
                .build()
try: 
    _, _ = evaluateModel(lr, paramGrid, "LR_Model", trainSet, testSet, timeSignature, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logs_dir=logs_dir,models_dir=models_dir)
except Exception as e:
    write2file(logs_dir,"Error:", str(e))
    print(e)

Fitting model..
Predicting on testSet..
Evaluating predictions..
Saving into: models/2020-05-05_01-03-20/LR_Model
Saving through format 2
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|    2|(1267,[0,1,2,3,4,...|
|       2.0|    2|(1267,[0,1,2,3,4,...|
|       2.0|    2|(1267,[0,1,2,3,4,...|
|       2.0|    2|(1267,[0,1,2,3,4,...|
|       2.0|    2|(1267,[0,1,2,3,4,...|
+----------+-----+--------------------+
only showing top 5 rows

Accuracy:  0.7685628819876815 
Parameters
 {Param(parent='LogisticRegression_cfa0f73f3e2f', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_cfa0f73f3e2f', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_cfa0f73f3e2f', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0

## Decision tree

In [28]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="selectedFeatures") 

paramGrid = ParamGridBuilder().build()

_, _ = evaluateModel(dt, paramGrid, "DT_Model", trainSet, testSet, timeSignature, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logs_dir=logs_dir,models_dir=models_dir)

Fitting model..


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 42964)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/socketserver.py", line 316, in _ha

Py4JError: An error occurred while calling o6263.evaluate

## Random forest

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="selectedFeatures") #numTrees=10
paramGrid = ParamGridBuilder() \
                .addGrid(rf.numTrees,[10,15,20,25]) \
                .build()

_, _ = evaluateModel(rf, paramGrid, "RF_Model", trainSet, testSet, timeSignature, evaluator=MulticlassClassificationEvaluator(),k=10, seed=None,logs_dir=logs_dir,models_dir=models_dir)

In [ ]:
write2file("logs_dir","Program finished!", "")

In [ ]:
write2file(model_indexing_dir,"New model folder created", \
           "Model finished: Yes" + \
           "Folder name: " + timeSignature + \
           "State: "+ filter_state + \
           "Logs directory: " + logs_dir +  \
           "File: " + file + \
           "Note: " + "ML - Test - No Quantilization - Dont use in production!" \
          )

## Hyperparameter tuning of best model

Since there is no predefined way to test multiple algorithms this is used!

In [ ]:
#models = createParams(models)
#bestModels = evaluateAlgorithm(models,trainSet)
#finalModel = getBestModel(bestModels,testSet)

#prediction = makePredictions(finalModel, testSet)
trainingSummary = finalModel.summary

# Make predictions on test documents. cvModel uses the best model found (lrModel).


#trainingSummary = cvModel.bestModel.stages[-1].summary
#selected = prediction.select("label", "probability", "prediction")
#selected.show(10)

In [ ]:
#finalModel = getBestModel(bestModels,testSet)
#print(finalModel)

In [ ]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
falsePositiveRateLabel = trainingSummary.falsePositiveRateByLabel
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
iterations = trainingSummary.totalIterations

print("Iterations: %s\nAccuracy: %s\nFPR: %s\nTPR: %s\nFPRL: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (iterations,accuracy, falsePositiveRate, truePositiveRate, falsePositiveRateLabel, fMeasure, precision, recall))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = prediction.select(['label']).collect()
y_pred = prediction.select(['prediction']).collect()

print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)